# Collecting Data from the Spotify Web API using Spotipy

## About the Spotipy Library:

From the [official Spotipy docs](https://spotipy.readthedocs.io/en/latest/): 
>"Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy you get full access to all of the music data provided by the Spotify platform."


## About using the Spotify Web API:

Spotify offers a number of [API endpoints](https://beta.developer.spotify.com/documentation/web-api/reference/) to access the Spotify data. In this notebook, I used the following:

- [search endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/search/search/) to get the track IDs 
- [audio features endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) to get the corresponding audio features.

The data was collected on several days during the months of April, May and August 2018.


## Goal of this notebook:

The goal is to show how to collect audio features data for tracks from the [official Spotify Web API](https://beta.developer.spotify.com/documentation/web-api/) in order to use it for further analysis/ machine learning which will be part of another notebook.

# 1. Setting Up

The below code is sufficient to set up Spotipy for querying the API endpoint. A more detailed explanation of the whole procedure is available in the [official docs](https://spotipy.readthedocs.io/en/latest/#installation).

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="0e41aa39d3bd452483da14e94524ce0f" 
secret = "6dd5e5ff0a6f4eac9f7ba9efc4312606"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# 2. Get the Track ID Data

The data collection is divided into 2 parts: the track IDs and the audio features. In this step, I'm going to collect 10.000 track IDs from the Spotify API.

The [search endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/search/search/) used in this step had a few limitations:

- limit: a maximum of 50 results can be returned per query
- offset: this is the index of the first result to return, so if you want to get the results with the index 50-100 you will need to set the offset to 50 etc.

Spotify cut down the maximum offset to 10.000 (as of May 2018?), I was lucky enough to do my first collection attempt while it was still 100.000

My solution: using a nested for loop, I increased the offset by 50 in the outer loop until the maxium limit/ offset was reached. The inner for loop did the actual querying and appending the returned results to appropriate lists which I used afterwards to create my dataframe.

In [2]:
import datetime
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
num_artists = []
explicit = []
release_month = []
release_weekday = []
artist_followers = []
artist_popularity = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist = sp.artist(t['artists'][0]['id'])
        artist_followers.append(artist['followers']['total'])
        artist_popularity.append(artist['popularity'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
        num_artists.append(len(t['artists']))
        explicit.append(t['explicit'])
        date = datetime.datetime.strptime(t['album']['release_date'], '%Y-%m-%d')
        release_month.append(date.month)
        release_weekday.append(date.isoweekday())
        
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 2385.7224082850007


# 3. EDA + Data Preparation

In the next few cells, I'm going to do some exploratory data analysis as well as data preparation of the newly gained data.

A quick check for the track_id list:

In [3]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


Looks good. Now loading the lists in a dataframe.

In [5]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity, 'num_artists': num_artists, 'release_month': release_month, 'release_weekday': release_weekday,  'artist_followers' : artist_followers, 'artist_popularity': artist_popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 9)


,artist_name,track_name,track_id,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,99,1,9,5,19671232,100
1,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,98,1,10,5,7153061,97
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,94,2,10,5,6183091,94
3,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,99,1,10,3,17952498,93
4,blackbear,hot girl bummer,7aiClxsDWFRQ0Kzk5KI5ku,97,1,8,5,2549258,89


In [6]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
artist_name          10000 non-null object
track_name           10000 non-null object
track_id             10000 non-null object
popularity           10000 non-null int64
num_artists          10000 non-null int64
release_month        10000 non-null int64
release_weekday      10000 non-null int64
artist_followers     10000 non-null int64
artist_popularity    10000 non-null int64
dtypes: int64(6), object(3)
memory usage: 703.2+ KB


Sometimes, the same track is returned under different track IDs (single, as part of an album etc.).

This needs to be checked for and corrected if needed.

In [7]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

515

There are 524 duplicate entries which will be dropped in the next cell:

In [8]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [9]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

This time the results are empty. Another way of checking this:

In [10]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name          0
track_name           0
track_id             0
popularity           0
num_artists          0
release_month        0
release_weekday      0
artist_followers     0
artist_popularity    0
dtype: int64

Checking how many tracks are left now:

In [11]:
df_tracks.shape

(9428, 9)

# 4: Get the Audio Features

With the [audio features endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) I will now get the audio features data for my 9460 track IDs.

The limitation for this endpoint is that a maximum of 100 track IDs can be submitted per query.

Again, I used a nested for loop. This time the outer loop was pulling track IDs in batches of size 100 and the inner for loop was doing the query and appending the results to the rows list.

Additionaly, I had to implement a check when a track ID didn't return any audio features (i.e. None was returned) as this was causing issues.

In [13]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 736
Time to run this code (in seconds): 15.215694664002513


# 5. EDA + Data Preparation

Same as with the first dataset, checking how the rows list looks like:

In [14]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 8692


Finally, I will load the audio features in a dataframe.

In [15]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (8692, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,audio_features,21jGcNKet2qwijlDFuPiPb,spotify:track:21jGcNKet2qwijlDFuPiPb,https://api.spotify.com/v1/tracks/21jGcNKet2qw...,https://api.spotify.com/v1/audio-analysis/21jG...,215280,4
1,0.598,0.427,7,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,audio_features,3eekarcy7kvN4yt5ZFzltW,spotify:track:3eekarcy7kvN4yt5ZFzltW,https://api.spotify.com/v1/tracks/3eekarcy7kvN...,https://api.spotify.com/v1/audio-analysis/3eek...,175721,4
2,0.474,0.631,5,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,audio_features,6Gg1gjgKi2AK4e0qzsR7sd,spotify:track:6Gg1gjgKi2AK4e0qzsR7sd,https://api.spotify.com/v1/tracks/6Gg1gjgKi2AK...,https://api.spotify.com/v1/audio-analysis/6Gg1...,189323,4
3,0.505,0.340,4,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,audio_features,1HfMVBKM75vxSfsQ5VefZ5,spotify:track:1HfMVBKM75vxSfsQ5VefZ5,https://api.spotify.com/v1/tracks/1HfMVBKM75vx...,https://api.spotify.com/v1/audio-analysis/1HfM...,206459,4
4,0.778,0.559,6,-7.109,0,0.0776,0.1280,0.000000,0.3990,0.6780,129.989,audio_features,7aiClxsDWFRQ0Kzk5KI5ku,spotify:track:7aiClxsDWFRQ0Kzk5KI5ku,https://api.spotify.com/v1/tracks/7aiClxsDWFRQ...,https://api.spotify.com/v1/audio-analysis/7aiC...,185093,1


In [16]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 18 columns):
danceability        8692 non-null float64
energy              8692 non-null float64
key                 8692 non-null int64
loudness            8692 non-null float64
mode                8692 non-null int64
speechiness         8692 non-null float64
acousticness        8692 non-null float64
instrumentalness    8692 non-null float64
liveness            8692 non-null float64
valence             8692 non-null float64
tempo               8692 non-null float64
type                8692 non-null object
id                  8692 non-null object
uri                 8692 non-null object
track_href          8692 non-null object
analysis_url        8692 non-null object
duration_ms         8692 non-null int64
time_signature      8692 non-null int64
dtypes: float64(9), int64(4), object(5)
memory usage: 1.2+ MB


Some columns are not needed for the analysis so I will drop them.

Also the ID column will be renamed to track_id so that it matches the column name from the first dataframe.

In [17]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(8692, 14)

In [18]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (8692, 14)


,artist_name,track_name,track_id,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,99,1,9,5,19671232,100,0.695,...,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,215280,4
1,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,98,1,10,5,7153061,97,0.598,...,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,175721,4
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,94,2,10,5,6183091,94,0.474,...,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,189323,4
3,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,99,1,10,3,17952498,93,0.505,...,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,206459,4
4,blackbear,hot girl bummer,7aiClxsDWFRQ0Kzk5KI5ku,97,1,8,5,2549258,89,0.778,...,-7.109,0,0.0776,0.1280,0.000000,0.3990,0.6780,129.989,185093,1


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8692 entries, 0 to 8691
Data columns (total 22 columns):
artist_name          8692 non-null object
track_name           8692 non-null object
track_id             8692 non-null object
popularity           8692 non-null int64
num_artists          8692 non-null int64
release_month        8692 non-null int64
release_weekday      8692 non-null int64
artist_followers     8692 non-null int64
artist_popularity    8692 non-null int64
danceability         8692 non-null float64
energy               8692 non-null float64
key                  8692 non-null int64
loudness             8692 non-null float64
mode                 8692 non-null int64
speechiness          8692 non-null float64
acousticness         8692 non-null float64
instrumentalness     8692 non-null float64
liveness             8692 non-null float64
valence              8692 non-null float64
tempo                8692 non-null float64
duration_ms          8692 non-null int64
time_signat

Just in case, checking for any duplicate tracks:

In [21]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


# 8. Adding numeric features
We now add some features to the data frame which will be useful for classifiers.

In [22]:
df['track_len'] = df.apply(lambda row: len(row['track_name']), axis=1)
df['artist_len'] = df.apply(lambda row: len(row['artist_name']), axis=1)

df.head()

,artist_name,track_name,track_id,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_len,artist_len
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,99,1,9,5,19671232,100,0.695,...,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,215280,4,7,11
1,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,98,1,10,5,7153061,97,0.598,...,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,175721,4,19,12
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,94,2,10,5,6183091,94,0.474,...,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,189323,4,40,10
3,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,99,1,10,3,17952498,93,0.505,...,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,206459,4,19,12
4,blackbear,hot girl bummer,7aiClxsDWFRQ0Kzk5KI5ku,97,1,8,5,2549258,89,0.778,...,0.0776,0.1280,0.000000,0.3990,0.6780,129.989,185093,1,15,9


# 7. Save to file
Everything seems to be fine so I will save the dataframe as a .csv file.

In [23]:
df.to_csv('../datasets/SpotifyAudioFeatures-full-29Nov.csv')